In [90]:
import pandas
import pandas as pd
import numpy as np
##!pip install gdown



# Airbnb Berlin Data Preparation Script

This script is designed to prepare and clean a dataset of Airbnb listings in Berlin, transforming and filtering the data to make it suitable for further analysis or modeling.


### 1. Data Download
- **Download CSV File**: The script uses `gdown` to download the Airbnb Berlin dataset from Google Drive and saves it as `Airbnb Berlin.csv`.

### 2. Data Cleaning and Transformation
- **Date Conversion**: Converts specified columns to datetime format to handle date-related operations.
  - Columns: `review_date`, `Host Since`, `First Review`, `Last Review`.
  
- **Boolean to Binary Conversion**: Converts boolean columns to binary (1/0) format.
  - Columns: `Is Superhost`, `Is Exact Location`.

- **Postal Code Cleaning**: Cleans and converts the `Postal Code` column to an integer format, removing any non-numeric characters.

- **Host Response Rate Conversion**: Converts `Host Response Rate` from a percentage string to a float value.

- **Column Drop**: Removes unnecessary columns to streamline the dataset.
  - Dropped Columns: `Country`, `Business Travel Ready`, `Is Superhost`, `Is Exact Location`, `Instant Bookable`, `Listing URL`, `Host URL`, `Postal Code`, `Square Feet`.

### 3. Feature Engineering
- **Top 10 Neighbourhoods**: Identifies the top 10 most frequent neighbourhoods and creates a new column `Top10Neighbourhood` to categorize listings.

- **Property Type Hierarchical Mapping**: Maps `Property Type` to higher-level hierarchical categories and creates a new column `Property Type_groups`.

- **String Conversion**: Converts specified columns to string type to ensure consistency.
  - Columns: `Reviewer Name`, `Listing Name`, `Host Name`, `City`, `Country Code`, `Top10Neighbourhood`, `neighbourhood`, `Neighborhood Group`, `Host Response Time`, `Property Type`, `Property Type_groups`, `Room Type`.

- **Price Cleaning**: Cleans the `Price` column by removing non-numeric characters and converting it to a float.

- **Handling Negative Values**: Replaces negative values with `NaN` in specific columns.
  - Columns: `Accomodates`, `Bathrooms`, `Bedrooms`, `Beds`, `Min Nights`.

### 4. Data Preparation for Analysis
- **Panel Data Creation**: Extracts and organizes relevant columns into a DataFrame (`df_panel`) for further analysis.
  - Selected Columns: `Listing ID`, `Listing Name`, `Host ID`, `Host Name`, `Host Since`, `Host Response Time`, `Host Response Rate`, `neighbourhood`, `Neighborhood Group`, `City`, `Country Code`, `Latitude`, `Longitude`, `Property Type`, `Room Type`, `Accomodates`, `Bathrooms`, `Bedrooms`, `Beds`, `Price`, `Guests Included`, `Min Nights`, `Reviews`, `First Review`, `Last Review`, `Overall Rating`, `Accuracy Rating`, `Cleanliness Rating`, `Checkin Rating`, `Communication Rating`, `Location Rating`, `Value Rating`, `Is Superhost_ind`, `Is Exact Location_ind`, `Postal Code_n`, `Top10Neighbourhood`, `Property Type_groups`.

- **Reviews Data Extraction**: Extracts review-related columns into a separate DataFrame (`df_reviews`).
  - Selected Columns: `Review ID`, `review_date`, `Reviewer ID`, `Reviewer Name`, `Comments`, `Listing ID`, `Listing Name`.

### 5. Data Filtering
- **Filter for Apartments**: Filters the panel DataFrame to include only listings categorized as `Apartment`.

### 6. Saving Data
- **Save as Pickle**: Saves the cleaned and prepared DataFrames (`df_panel`, `df_reviews`) as pickle files for future use.

## Output Files
- `Airbnb_Berlin.pkl`: Pickled version of the entire cleaned DataFrame.
- `df_panel.pkl`: Pickled version of the panel DataFrame focusing on apartments.
- `df_reviews.pkl`: Pickled version of the reviews DataFrame.



In [63]:
##https://drive.google.com/file/d/1U4_zwxQSGMG1-EHwsb8K1WagQ0IjNNLP/view?usp=drive_link
import gdown
download_url='https://drive.google.com/uc?id=1U4_zwxQSGMG1-EHwsb8K1WagQ0IjNNLP'
output = 'Airbnb Berlin.csv'  # The name you want to save the file as

gdown.download(download_url, output, quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1U4_zwxQSGMG1-EHwsb8K1WagQ0IjNNLP
From (redirected): https://drive.google.com/uc?id=1U4_zwxQSGMG1-EHwsb8K1WagQ0IjNNLP&confirm=t&uuid=8ca84a7d-73ed-4520-91e2-d706717e7923
To: C:\Users\Gil\Documents\GitHub\Berlin-Airbnb-Ratings--Predict-listing-prices-\Airbnb Berlin.csv
100%|███████████████████████████████████████████████████████████████████████████████| 264M/264M [00:27<00:00, 9.57MB/s]


'Airbnb Berlin.csv'

In [64]:

# Replace 'your_file.csv' with the path to your CSV file
file_path = 'Airbnb Berlin.csv'

# Read the CSV file
df = pd.read_csv(file_path)
df.to_pickle('Airbnb_Berlin.pkl')
df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
# Display the first few rows of the DataFrame
##print(df.head())
#df.info()
#df['Host Since']

# Convert date columns to datetime
date_columns = ['review_date', 'Host Since', 'First Review', 'Last Review']
for col in date_columns:
    df[col] = pd.to_datetime(df[col], format='%m-%d-%y', errors='coerce')

# Convert boolean columns to binary
boolean_columns = ['Is Superhost', 'Is Exact Location']
for col in boolean_columns:
    df[f'{col}_ind'] = df[col].map({'t': 1, 'f': 0, None: np.nan}).astype('Int64')
    
#df['Is Superhost_ind'] = df['Is Superhost_ind'].astype('int64')

# Clean 'Postal Code' column and convert to integer
df['Postal Code_n'] = df['Postal Code'].str.replace('[^0-9]', '', regex=True)
df[['Review ID', 'Reviewer ID', 'Postal Code_n']] = df[['Review ID', 'Reviewer ID', 'Postal Code_n']].astype('Int64')

# Convert 'Host Response Rate' to float
df['Host Response Rate'] = df['Host Response Rate'].str.replace('%', '').astype(float) / 100

# Drop unnecessary columns
df = df.drop(columns=['Country', 'Business Travel Ready', 'Is Superhost', 'Is Exact Location','Instant Bookable', 'Listing URL', 'Host URL', 'Postal Code','Square Feet'])

# Create a new column for the top 10 most frequent neighbourhoods
top_10_neighbourhoods = df['neighbourhood'].value_counts().nlargest(10).index
df['Top10Neighbourhood'] = df['neighbourhood'].apply(lambda x: x if x in top_10_neighbourhoods else 'Other')

# Define hierarchical categories for 'Property Type'
Property_Type_mapping = {
    'Apartment': 'Apartment',
    'Loft': 'Apartment',
    'Condominium': 'Apartment',
    'Serviced apartment': 'Apartment',
    'House': 'House',
    'Guest suite': 'Guesthouse',
    'Townhouse': 'House',
    'Guesthouse': 'Guesthouse',
    'Bed and breakfast': 'Guesthouse',
    'Hostel': 'Guesthouse',
    'Boutique hotel': 'Hotel',
    'Other': 'Other',
    'Bungalow': 'House',
    'Houseboat': 'House',
    'Hotel': 'Hotel',
    'Cabin': 'House',
    'Cottage': 'House',
    'Villa': 'House',
    'Tiny house': 'House',
    'Boat': 'Other',
    'Aparthotel': 'Hotel',
    'Castle': 'Hotel',
    'Resort': 'Hotel',
    'Tent': 'Other',
    'Hut': 'Other',
    'Camper/RV': 'Other',
    'Cave': 'Other',
    'Train': 'Other',
    'Barn': 'Other',
    'Casa particular (Cuba)': 'Other',
    'Pension (South Korea)': 'Other',
    'Earth house': 'Other',
    'Tipi': 'Other',
    'Treehouse': 'Other',
    'Vacation home': 'House'
}

df['Property Type_groups'] = df['Property Type'].map(Property_Type_mapping)

# Convert string columns to string type
string_columns = ['Reviewer Name', 'Listing Name', 'Host Name', 'City', 'Country Code', 'Top10Neighbourhood', 'neighbourhood', 'Neighborhood Group', 'Host Response Time', 'Property Type', 'Property Type_groups', 'Room Type']
df[string_columns] = df[string_columns].astype('string')

# Replace values below 0 with NaN in the specified columns
columns_to_update = ['Accomodates', 'Bathrooms', 'Bedrooms', 'Beds', 'Min Nights']

for col in columns_to_update:
    df.loc[df[col] < 0, col] = np.nan





# Ensure the 'Price' column is of type string, remove non-numeric characters, and convert to float
df['Price'] = df['Price'].astype(str)
df['Price'] = df['Price'].str.replace(r'[^\d.]', '', regex=True)
df['Price'] = df['Price'].apply(lambda x: np.nan if x == '' else x).astype('float64')


C:\Users\Gil\AppData\Local\Temp\ipykernel_18772\4088954857.py:5: DtypeWarning: Columns (19,32) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


In [65]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 456961 entries, 0 to 456960
Data columns (total 43 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   index                  456961 non-null  int64         
 1   Review ID              452805 non-null  Int64         
 2   review_date            452805 non-null  datetime64[ns]
 3   Reviewer ID            452805 non-null  Int64         
 4   Reviewer Name          452805 non-null  string        
 5   Comments               452595 non-null  object        
 6   Listing ID             456961 non-null  int64         
 7   Listing Name           456756 non-null  string        
 8   Host ID                456961 non-null  int64         
 9   Host Name              456913 non-null  string        
 10  Host Since             456913 non-null  datetime64[ns]
 11  Host Response Time     398194 non-null  string        
 12  Host Response Rate     398194 non-null  floa

In [66]:
df_panel= df[[ 
    'Listing ID', 
    'Listing Name', 
    'Host ID', 
    'Host Name',
    'Host Since', 
    'Host Response Time', 
    'Host Response Rate',
    'neighbourhood', 
    'Neighborhood Group', 
    'City', 
    'Country Code',
    'Latitude', 
    'Longitude', 
    'Property Type', 
    'Room Type', 
    'Accomodates',
    'Bathrooms', 
    'Bedrooms', 
    'Beds', 
    'Price',
    'Guests Included', 
    'Min Nights', 
    'Reviews', 
    'First Review',
    'Last Review', 
    'Overall Rating', 
    'Accuracy Rating',
    'Cleanliness Rating', 
    'Checkin Rating', 
    'Communication Rating',
    'Location Rating', 
    'Value Rating', 
    'Is Superhost_ind',
    'Is Exact Location_ind', 
    'Postal Code_n',
    'Top10Neighbourhood', 
    'Property Type_groups']]



df_reviews=df[[ 'Review ID', 
    'review_date', 
    'Reviewer ID', 
    'Reviewer Name',
    'Comments', 
    'Listing ID', 
    'Listing Name', 
]]

In [67]:
# Save DataFrame to a pickle file
df_reviews.to_pickle('df_reviews.pkl')

In [68]:
df_panel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 456961 entries, 0 to 456960
Data columns (total 37 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   Listing ID             456961 non-null  int64         
 1   Listing Name           456756 non-null  string        
 2   Host ID                456961 non-null  int64         
 3   Host Name              456913 non-null  string        
 4   Host Since             456913 non-null  datetime64[ns]
 5   Host Response Time     398194 non-null  string        
 6   Host Response Rate     398194 non-null  float64       
 7   neighbourhood          456961 non-null  string        
 8   Neighborhood Group     456961 non-null  string        
 9   City                   456928 non-null  string        
 10  Country Code           456961 non-null  string        
 11  Latitude               456961 non-null  float64       
 12  Longitude              456961 non-null  floa

In [69]:
df_reviews.info()
#side_table

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 456961 entries, 0 to 456960
Data columns (total 7 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   Review ID      452805 non-null  Int64         
 1   review_date    452805 non-null  datetime64[ns]
 2   Reviewer ID    452805 non-null  Int64         
 3   Reviewer Name  452805 non-null  string        
 4   Comments       452595 non-null  object        
 5   Listing ID     456961 non-null  int64         
 6   Listing Name   456756 non-null  string        
dtypes: Int64(2), datetime64[ns](1), int64(1), object(1), string(2)
memory usage: 25.3+ MB


In [82]:

#property_type_percentages = df['Property Type_groups'].value_counts(normalize=True) * 100
#average_price_per_group = df.groupby('Property Type_groups')['Price'].mean()
# Calculate the average price for each property group

# Filter the DataFrame to include only apartments
df_panel = df_panel[df_panel['Property Type'] == 'Apartment']

# Note: The purpose of this filter is to focus the research population on Airbnb apartments only.


In [83]:
df_panel.shape
df_panel.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12583 entries, 65536 to 456958
Data columns (total 37 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Listing ID             12583 non-null  int64         
 1   Listing Name           12560 non-null  string        
 2   Host ID                12583 non-null  int64         
 3   Host Name              12572 non-null  string        
 4   Host Since             12572 non-null  datetime64[ns]
 5   Host Response Time     7307 non-null   string        
 6   Host Response Rate     7307 non-null   float64       
 7   neighbourhood          12583 non-null  string        
 8   Neighborhood Group     12583 non-null  string        
 9   City                   12580 non-null  string        
 10  Country Code           12583 non-null  string        
 11  Latitude               12583 non-null  float64       
 12  Longitude              12583 non-null  float64       
 13  P

In [84]:
# Save DataFrame to a pickle file

df_panel.to_pickle('df_panel.pkl')

In [85]:
df_panel

,Listing ID,Listing Name,Host ID,Host Name,Host Since,Host Response Time,Host Response Rate,neighbourhood,Neighborhood Group,City,...,Cleanliness Rating,Checkin Rating,Communication Rating,Location Rating,Value Rating,Is Superhost_ind,Is Exact Location_ind,Postal Code_n,Top10Neighbourhood,Property Type_groups
65536,13761071,Central apartment near Alex (Apt. 9),80675177,Sven,2016-06-29,within a few hours,1.0,Mitte,Mitte,Berlin,...,10.0,10.0,10.0,10.0,9.0,1,1,10179,Mitte,Apartment
65603,13763834,"SchÃ¶ne, helle Whg nahe Tempelh. Feld in NeukÃ...",80714211,Renald,2016-06-29,<NA>,NaN,NeukÃ¶lln,NeukÃ¶lln,Berlin,...,9.0,10.0,10.0,9.0,10.0,0,1,12051,NeukÃ¶lln,Apartment
65625,13764102,Cozy and quiet Loft in Central Berlin,80179711,Sonja,2016-06-27,<NA>,NaN,Mitte,Mitte,Berlin,...,NaN,NaN,NaN,NaN,NaN,0,1,10119,Mitte,Apartment
65626,13764166,"Cozy room in Berlin, Charlottenburg!",62639492,Eva,2016-03-12,<NA>,NaN,Charlottenburg,Charlottenburg-Wilm.,Berlin,...,10.0,10.0,10.0,10.0,10.0,0,1,10589,Charlottenburg,Apartment
65627,13765505,Small but nice and quite room,49985648,Janka,2015-11-27,<NA>,NaN,Prenzlauer Berg,Pankow,Berlin,...,10.0,10.0,10.0,10.0,10.0,0,0,10407,Prenzlauer Berg,Apartment
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456582,9991497,"Chillout-Oase fÃ¼r ""GroÃ und Klein""",44816934,Luca,2015-09-22,<NA>,NaN,NiederschÃ¶neweide,Treptow - KÃ¶penick,Berlin,...,NaN,NaN,NaN,NaN,NaN,0,0,<NA>,Other,Apartment
456585,9993168,GemÃ¼tliches Zimmer zur Ãbernachtung,40660291,Anh,2015-08-06,<NA>,NaN,Rummelsburg,Lichtenberg,Berlin,...,10.0,10.0,9.0,9.0,9.0,0,0,<NA>,Other,Apartment
456605,9994644,Potsdamer Platz-Top Spot Studio 1,22276081,Martin,2014-10-08,within an hour,1.0,Kreuzberg,Friedrichshain-Kreuzberg,Berlin,...,10.0,10.0,10.0,10.0,10.0,1,1,<NA>,Kreuzberg,Apartment
456954,999465,Familyhome for your vacation,5494219,Simone,2013-03-17,<NA>,NaN,Friedrichshain,Friedrichshain-Kreuzberg,Berlin,...,10.0,10.0,10.0,10.0,10.0,0,0,<NA>,Friedrichshain,Apartment
